In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [4]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

In [5]:
# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

In [6]:
# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

In [7]:
# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

In [8]:
# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [9]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=laptop"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

In [10]:
amazon_df = pd.DataFrame.from_dict(d)
amazon_df.replace({'title': ''}, np.nan, inplace=True)
amazon_df = amazon_df.dropna(subset=['title'])
amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [11]:
amazon_df

,title,price,rating,reviews,availability
0,"ACEMAGIC 2025 LX15 Laptop, 15.6-inch FHD (1920...",,4.6 out of 5 stars,25 ratings,In Stock
1,"AOC 2025 AX15 Laptop Computer, 15.6-inch FHD S...",,4.3 out of 5 stars,5 ratings,
2,"HP 14 Laptop, Intel Celeron N4020, 4 GB RAM, 6...",,4.3 out of 5 stars,"3,741 ratings",Only 4 left in stock - order soon.
3,"HP Stream 14"" HD BrightView Laptop, Intel Cele...",,4.5 out of 5 stars,"2,999 ratings",In Stock
4,Apple 2025 MacBook Air 13-inch Laptop with M4 ...,,4.8 out of 5 stars,"1,782 ratings",Not Available
5,"SGIN 17.3 Inch Laptop Computer, Win 11 Laptops...",,4.3 out of 5 stars,25 ratings,Only 6 left in stock - order soon.
6,Lenovo IdeaPad 5X 2-in-1 Copilot+ Extra-Long B...,,4.0 out of 5 stars,26 ratings,In Stock
7,"2025 AMD Laptop, 15.6inch Laptop Computer with...",,4.6 out of 5 stars,24 ratings,In Stock
8,"ASUS Vivobook 16 Laptop, Copilot+ PC, 16” WUXG...",,4.5 out of 5 stars,129 ratings,In Stock
9,"15.6 Inch Laptop-Computer with MS Office, 12th...",,4.5 out of 5 stars,15 ratings,In Stock
